In [67]:
from email.feedparser import headerRE

import seaborn as sns
import matplotlib.pyplot as plt
import polars as pl
import polars.selectors as cs
import altair as alt
import plotly.express as px
import plotly.graph_objects as go
import great_tables as tg
import datetime as dt
import numpy as np
import pandas as pd

In [3]:
df_path = r"F:\Datasets\CSV datasets\Bank_Transaction_Fraud_Detection.csv"

In [4]:
df = pl.read_csv(df_path, try_parse_dates=True)

In [5]:
df = df.with_columns(
    pl.col('Account_Balance') * 0.011007299,
    pl.col('Transaction_Amount') * 0.011007299,
)

In [6]:
df = df.drop(
    'Customer_ID', 'Transaction_ID', 'Customer_Contact', 'Customer_Email',
    'Transaction_Currency', 'Transaction_Currency', 'Merchant_ID',
    'Customer_Name'
)

In [7]:
df['State'].value_counts()

State,count
str,u32
"""Telangana""",5952
"""Himachal Pradesh""",5875
"""Tamil Nadu""",5841
"""Goa""",5871
"""Nagaland""",6031
…,…
"""Madhya Pradesh""",5928
"""Delhi""",5943
"""Maharashtra""",5784


In [8]:
df.select('*')

Gender,Age,State,City,Bank_Branch,Account_Type,Transaction_Date,Transaction_Time,Transaction_Amount,Transaction_Type,Merchant_Category,Account_Balance,Transaction_Device,Transaction_Location,Device_Type,Is_Fraud,Transaction_Description
str,i64,str,str,str,str,date,time,f64,str,str,f64,str,str,str,i64,str
"""Male""",60,"""Kerala""","""Thiruvananthapuram""","""Thiruvananthapuram Branch""","""Savings""",2025-01-23,16:04:07,356.80655,"""Transfer""","""Restaurant""",820.674164,"""Voice Assistant""","""Thiruvananthapuram, Kerala""","""POS""",0,"""Bitcoin transaction"""
"""Female""",51,"""Maharashtra""","""Nashik""","""Nashik Branch""","""Business""",2025-01-11,17:14:53,480.167001,"""Bill Payment""","""Restaurant""",821.393931,"""POS Mobile Device""","""Nashik, Maharashtra""","""Desktop""",0,"""Grocery delivery"""
"""Male""",20,"""Bihar""","""Bhagalpur""","""Bhagalpur Branch""","""Savings""",2025-01-25,03:09:52,694.148454,"""Bill Payment""","""Groceries""",735.485595,"""ATM""","""Bhagalpur, Bihar""","""Desktop""",0,"""Mutual fund investment"""
"""Female""",57,"""Tamil Nadu""","""Chennai""","""Chennai Branch""","""Business""",2025-01-19,12:27:02,154.110111,"""Debit""","""Entertainment""",640.372515,"""POS Mobile App""","""Chennai, Tamil Nadu""","""Mobile""",0,"""Food delivery"""
"""Female""",43,"""Punjab""","""Amritsar""","""Amritsar Branch""","""Savings""",2025-01-30,18:30:46,201.820588,"""Transfer""","""Entertainment""",177.311736,"""Virtual Card""","""Amritsar, Punjab""","""Mobile""",0,"""Debt repayment"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Female""",55,"""Dadra and Nagar Haveli and Dam…","""Diu""","""Diu Branch""","""Business""",2025-01-08,18:42:09,1084.370192,"""Credit""","""Restaurant""",412.499741,"""Desktop/Laptop""","""Diu, Dadra and Nagar Haveli an…","""ATM""",0,"""ATM withdrawal"""
"""Male""",51,"""Manipur""","""Kangpokpi""","""Kangpokpi Branch""","""Business""",2025-01-01,20:51:21,446.825342,"""Withdrawal""","""Groceries""",583.796319,"""ATM""","""Kangpokpi, Manipur""","""ATM""",0,"""Subscription box"""
"""Female""",41,"""Chandigarh""","""Chandigarh""","""Chandigarh Branch""","""Savings""",2025-01-28,10:47:40,677.82617,"""Withdrawal""","""Health""",1059.181309,"""ATM""","""Chandigarh, Chandigarh""","""Desktop""",0,"""Charity donation"""


In [19]:
pl.DataFrame({
    "column": df.columns,
    "unique_count": [df[col].n_unique() for col in df.columns]
})

column,unique_count
str,i64
"""Gender""",2
"""Age""",53
"""State""",34
"""City""",145
"""Bank_Branch""",145
…,…
"""Transaction_Device""",20
"""Transaction_Location""",148
"""Device_Type""",4


In [30]:
df = df.with_columns(
    pl.col('Transaction_Date').dt.year().alias('Year'),
    pl.col('Transaction_Date').dt.month().alias('Month'),
    pl.col('Transaction_Date').dt.day().alias('Day'),
    pl.col('Transaction_Time').dt.hour().alias('Hour'),
    pl.col('Transaction_Time').dt.minute().alias('Minute'),
    pl.col('Transaction_Time').dt.second().alias('Second'),
)

In [31]:
df = df.drop(
    'Transaction_Date', 'Transaction_Time'
)

In [32]:
df

Gender,Age,State,City,Bank_Branch,Account_Type,Transaction_Amount,Transaction_Type,Merchant_Category,Account_Balance,Transaction_Device,Transaction_Location,Device_Type,Is_Fraud,Transaction_Description,Year,Month,Day,Hour,Minute,Second
str,i64,str,str,str,str,f64,str,str,f64,str,str,str,i64,str,i32,i8,i8,i8,i8,i8
"""Male""",60,"""Kerala""","""Thiruvananthapuram""","""Thiruvananthapuram Branch""","""Savings""",356.80655,"""Transfer""","""Restaurant""",820.674164,"""Voice Assistant""","""Thiruvananthapuram, Kerala""","""POS""",0,"""Bitcoin transaction""",2025,1,23,16,4,7
"""Female""",51,"""Maharashtra""","""Nashik""","""Nashik Branch""","""Business""",480.167001,"""Bill Payment""","""Restaurant""",821.393931,"""POS Mobile Device""","""Nashik, Maharashtra""","""Desktop""",0,"""Grocery delivery""",2025,1,11,17,14,53
"""Male""",20,"""Bihar""","""Bhagalpur""","""Bhagalpur Branch""","""Savings""",694.148454,"""Bill Payment""","""Groceries""",735.485595,"""ATM""","""Bhagalpur, Bihar""","""Desktop""",0,"""Mutual fund investment""",2025,1,25,3,9,52
"""Female""",57,"""Tamil Nadu""","""Chennai""","""Chennai Branch""","""Business""",154.110111,"""Debit""","""Entertainment""",640.372515,"""POS Mobile App""","""Chennai, Tamil Nadu""","""Mobile""",0,"""Food delivery""",2025,1,19,12,27,2
"""Female""",43,"""Punjab""","""Amritsar""","""Amritsar Branch""","""Savings""",201.820588,"""Transfer""","""Entertainment""",177.311736,"""Virtual Card""","""Amritsar, Punjab""","""Mobile""",0,"""Debt repayment""",2025,1,30,18,30,46
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Female""",55,"""Dadra and Nagar Haveli and Dam…","""Diu""","""Diu Branch""","""Business""",1084.370192,"""Credit""","""Restaurant""",412.499741,"""Desktop/Laptop""","""Diu, Dadra and Nagar Haveli an…","""ATM""",0,"""ATM withdrawal""",2025,1,8,18,42,9
"""Male""",51,"""Manipur""","""Kangpokpi""","""Kangpokpi Branch""","""Business""",446.825342,"""Withdrawal""","""Groceries""",583.796319,"""ATM""","""Kangpokpi, Manipur""","""ATM""",0,"""Subscription box""",2025,1,1,20,51,21
"""Female""",41,"""Chandigarh""","""Chandigarh""","""Chandigarh Branch""","""Savings""",677.82617,"""Withdrawal""","""Health""",1059.181309,"""ATM""","""Chandigarh, Chandigarh""","""Desktop""",0,"""Charity donation""",2025,1,28,10,47,40


In [33]:
X = df.select(cs.exclude('Is_Fraud'))

In [34]:
y = df.get_column('Is_Fraud')

In [40]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [38]:
X.select(cs.string()).columns

['Gender',
 'State',
 'City',
 'Bank_Branch',
 'Account_Type',
 'Transaction_Type',
 'Merchant_Category',
 'Transaction_Device',
 'Transaction_Location',
 'Device_Type',
 'Transaction_Description']

In [39]:
X.select(cs.numeric()).columns

['Age',
 'Transaction_Amount',
 'Account_Balance',
 'Year',
 'Month',
 'Day',
 'Hour',
 'Minute',
 'Second']

In [46]:
column_transformer = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(
            handle_unknown='ignore', sparse_output=False, drop='first'),
        X.select(cs.string()).columns),
        ('passthrough', 'passthrough', X.select(cs.numeric()).columns)
    ])

In [47]:
X_encoded = column_transformer.fit_transform(X)

In [50]:
pl.DataFrame(X_encoded).estimated_size('gb')

1.0162591934204102

In [52]:
X_encoded.shape

(200000, 682)

In [53]:
from sklearn.feature_extraction import FeatureHasher

In [75]:
categorical_cols = X.select(cs.string()).columns

In [76]:
X_pandas = X.select(categorical_cols).to_pandas()

In [77]:
hasher = FeatureHasher(n_features=128, input_type='string')

In [80]:
hashed_features = hasher.fit_transform(X_pandas.values.tolist())

In [81]:
hashed_features

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2085444 stored elements and shape (200000, 128)>

In [82]:
numerical_cols = X.select(cs.numeric()).columns

In [83]:
X_numeric = X.select(numerical_cols).to_pandas()

In [84]:
from sklearn.preprocessing import StandardScaler

In [85]:
scaler = StandardScaler()

In [89]:
scaled_numerical = scaler.fit_transform(X_numeric)

In [90]:
X_final = np.hstack([hashed_features.toarray(), scaled_numerical])

In [92]:
hashed_feature_names = [f'hashed_feature_{i}' for i in range(hashed_features.shape[1])]

In [93]:
numerical_feature_names = X.select(cs.numeric()).columns

In [94]:
final_column_names = hashed_feature_names + numerical_feature_names

In [95]:
X_final_df = pd.DataFrame(X_final, columns=final_column_names)

In [107]:
X_encoded = pl.DataFrame(data=X_final_df, schema=list(X_final_df.columns))

In [112]:
X_encoded.head(5)

hashed_feature_0,hashed_feature_1,hashed_feature_2,hashed_feature_3,hashed_feature_4,hashed_feature_5,hashed_feature_6,hashed_feature_7,hashed_feature_8,hashed_feature_9,hashed_feature_10,hashed_feature_11,hashed_feature_12,hashed_feature_13,hashed_feature_14,hashed_feature_15,hashed_feature_16,hashed_feature_17,hashed_feature_18,hashed_feature_19,hashed_feature_20,hashed_feature_21,hashed_feature_22,hashed_feature_23,hashed_feature_24,hashed_feature_25,hashed_feature_26,hashed_feature_27,hashed_feature_28,hashed_feature_29,hashed_feature_30,hashed_feature_31,hashed_feature_32,hashed_feature_33,hashed_feature_34,hashed_feature_35,hashed_feature_36,…,hashed_feature_100,hashed_feature_101,hashed_feature_102,hashed_feature_103,hashed_feature_104,hashed_feature_105,hashed_feature_106,hashed_feature_107,hashed_feature_108,hashed_feature_109,hashed_feature_110,hashed_feature_111,hashed_feature_112,hashed_feature_113,hashed_feature_114,hashed_feature_115,hashed_feature_116,hashed_feature_117,hashed_feature_118,hashed_feature_119,hashed_feature_120,hashed_feature_121,hashed_feature_122,hashed_feature_123,hashed_feature_124,hashed_feature_125,hashed_feature_126,hashed_feature_127,Age,Transaction_Amount,Account_Balance,Year,Month,Day,Hour,Minute,Second
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.045534,-0.599702,0.80729,0.0,0.0,0.862982,0.651949,-1.474207,-1.297654
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.456865,-0.207182,0.809676,0.0,0.0,-0.520739,0.796519,-0.895676,1.355197
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.570771,0.473684,0.524828,0.0,0.0,1.093603,-1.227458,-1.184941,1.297526
0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.849311,-1.24466,0.20946,0.0,0.0,0.401742,0.07367,-0.143587,-1.586007
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.066396,-1.092851,-1.325919,0.0,0.0,1.670153,0.941088,0.029973,0.951502


In [113]:
y.head(5)

Is_Fraud
i64
0
0
0
0
0
